In [ ]:
import pandas as pd
from datamodel import OrderDepth, TradingState, Order
from typing import List
import numpy as np


ModuleNotFoundError: No module named 'pandas'

In [2]:

combined_prices = pd.read_csv(r"C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\IMC Competition\combined_prices.csv")
combined_trades = pd.read_csv(r"C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\IMC Competition\combined_trades.csv")

In [7]:
combined_prices.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,source_file
0,0,0,RAINFOREST_RESIN,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0,prices_round_1_day_0.csv
1,0,0,KELP,2028,30,NaN,NaN,NaN,NaN,2032,30,NaN,NaN,NaN,NaN,2030.0,0.0,prices_round_1_day_0.csv
2,0,0,SQUID_INK,1968,30,NaN,NaN,NaN,NaN,1971,30,NaN,NaN,NaN,NaN,1969.5,0.0,prices_round_1_day_0.csv
3,0,100,SQUID_INK,1969,4,1967.0,31.0,NaN,NaN,1971,31,NaN,NaN,NaN,NaN,1970.0,0.0,prices_round_1_day_0.csv
4,0,100,KELP,2030,4,2028.0,31.0,NaN,NaN,2032,31,NaN,NaN,NaN,NaN,2031.0,0.0,prices_round_1_day_0.csv


In [ ]:
# step 1
class Trader:
    def run(self, state: TradingState):
        # This is where your trading logic will go.
        # For now, let's print some data to see what's inside the state.
        print("Trader data received:", state.traderData)
        print("Observations:", state.observations)
        
        # Prepare a dictionary to store orders for each product.
        result = {}
        
        # Step 2
        # Loop through each product available in the market orders.
        for product in state.order_depths:
            # Get the details (order depth) for that product.
            order_depth: OrderDepth = state.order_depths[product]
            # Prepare a list to hold our orders (if any) for this product.
            orders: List[Order] = []

            #  Step 3
            #  For simplicity, we set an acceptable price.
            acceptable_price = 10  
            print("Product:", product, "- Acceptable price:", acceptable_price)


            # Step 4
            # For Buying:
            # Check if there are any sell orders in the order depth. 
            # If yes, get the best sell price and its quantity. 
            # Then decide if the price is lower than your threshold.

            # Check for sell orders.
            if len(order_depth.sell_orders) != 0:
                # Get the best ask price and quantity (sell orders are stored in a dict).
                best_ask, best_ask_amount = list(order_depth.sell_orders.items())[0]
                if int(best_ask) < acceptable_price:
                    # In our simple strategy, we want to buy the full quantity available.
                    print("BUY", str(-best_ask_amount) + "x at", best_ask)
                    # The Order class takes (symbol, price, quantity)
                    # Quantity is negative for a buy order in this context.
                    orders.append(Order(product, best_ask, -best_ask_amount))


            # For Selling:
            # Likewise, inspect the buy orders and decide if you want to sell:
            # Check for buy orders
            if len(order_depth.buy_orders) != 0:
                best_bid, best_bid_amount = list(order_depth.buy_orders.items())[0]
                if int(best_bid) > acceptable_price:
                    print("SELL", str(best_bid_amount) + "x at", best_bid)
                    # For sell orders, the quantity is positive because you sell what you already own.
                    orders.append(Order(product, best_bid, -best_bid_amount))

            # Step 5
            # After checking both buy and sell conditions for a product, 
            # add your list of orders into the result dictionary using the 
            # product name as the key:
            # Save the orders (if any) for the current product.
            result[product] = orders
        
        # Step 6
        # Define a traderData string to pass some state information to the next iteration.
        traderData = "SAMPLE_STATE"

        # Step 7 : Setting Conversion Request
        # Sometimes you might need to place a conversion request based 
        # on your current position. For now, we can keep it simple:

        # Example conversion request variable. Some strategies might use this.
        conversions = 1  # or 0 if no conversion is needed

        # Return orders, conversion request, and traderData.
        return result, conversions, traderData